In [2]:
import pandas

In [5]:
conn_params = {
    "host": "127.0.0.1",  # e.g., "localhost"
    "dbname": "postgres",  # e.g., "mydatabase"
    "user": "postgres",  # e.g., "myusername"
    "password": "postgres",  # e.g., "mypassword"
    "port": "5432"  # default port for PostgreSQL
}

In [3]:
import psycopg
import pandas as pd

In [7]:
conn = psycopg.connect(**conn_params)

In [8]:
query = "select hdate, open, high, low, close, volume, trade_count   from turtle.bars_history bh where symbol = 'AMZN' and hdate > '2024-07-01' order by hdate"

In [9]:
with psycopg.connect(**conn_params) as conn:
    with conn.cursor() as cur:
        cur.execute(query)
        # Fetch all rows
        rows = cur.fetchall()
        # Get column names from the cursor description
        col_names = [desc.name for desc in cur.description]

dtypes = {
    "hdate": "string",
    "open": "float64",
    "high": "float64",
    "low": "float64",
    "close": "float64",
    "volume": "int64",
    "trade_count": "int64"
}

# Create a pandas DataFrame from the fetched data
df = pd.DataFrame(rows, columns=col_names).astype(dtypes)
df['hdate'] = pd.to_datetime(df['hdate'])
df = df.set_index(['hdate'])

# Display the DataFrame
print(df)

               open      high       low   close     volume  trade_count
hdate                                                                  
2024-07-02  197.280  200.4300  195.9300  200.00   45600013       485437
2024-07-03  199.940  200.0290  196.7601  197.59   31597926       395364
2024-07-05  198.650  200.5500  198.1700  200.00   39858885       451244
2024-07-08  200.040  201.2000  197.9600  199.29   34767261       443701
2024-07-09  199.400  200.5700  199.0450  199.34   32756736       379497
2024-07-10  199.995  200.1100  197.6900  199.79   32883753       414183
2024-07-11  200.090  200.2699  192.8600  195.05   44565041       582321
2024-07-12  194.800  196.4700  193.8301  194.49   30598525       427388
2024-07-15  194.560  196.1900  190.8300  192.72   40683227       571566
2024-07-16  195.590  196.6200  192.2400  193.02   33994714       449302
2024-07-17  191.350  191.5800  185.9900  187.93   48076139       682876
2024-07-18  189.590  189.6800  181.4480  183.75   51043626      

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 28 entries, 2024-07-02 to 2024-08-09
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   open         28 non-null     float64
 1   high         28 non-null     float64
 2   low          28 non-null     float64
 3   close        28 non-null     float64
 4   volume       28 non-null     int64  
 5   trade_count  28 non-null     int64  
dtypes: float64(4), int64(2)
memory usage: 1.5 KB


In [2]:
import pandas_ta as ta

In [1]:
import sys
sys.path.insert(0, "/home/jack/github/turtle-backtest")
print(sys.path)

['/home/jack/github/turtle-backtest', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/home/jack/github/turtle-backtest/.venv/lib/python3.12/site-packages', '/home/jack/github/turtle-backtest']


NameError: name 'sys' is not defined

In [1]:
import os
os.chdir('/home/jack/github/turtle-backtest')
from dotenv import load_dotenv
load_dotenv()
print(os.getenv("ALPACA_API_KEY"))

AKI8X4PEAZE996RQWLJC


In [5]:
import os
from datetime import datetime
from alpaca.data.enums import DataFeed
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.requests import StockBarsRequest
from alpaca.data.historical import StockHistoricalDataClient

stock_data_client = StockHistoricalDataClient(os.getenv("ALPACA_API_KEY"), os.getenv("ALPACA_SECRET_KEY"))

start_date: datetime = datetime(year=2024, month=8, day=23)
end_date: datetime = datetime(year=2024, month=8, day=30)  
symbol="AMZN"
request = StockBarsRequest(
            symbol_or_symbols=symbol,
            start=start_date,
            end=end_date,
            limit=10000,
            timeframe=TimeFrame(1, TimeFrameUnit.Day),
            feed=DataFeed.SIP,
        )
data = stock_data_client.get_stock_bars(request_params=request)
print(data)
print(type(data))
print(type(data[symbol]))

bars = data[symbol]
print(type(bars[0].high))
print(type(bars[0]))

data={'AMZN': [{   'close': 177.04,
    'high': 178.9699,
    'low': 175.24,
    'open': 177.34,
    'symbol': 'AMZN',
    'timestamp': datetime.datetime(2024, 8, 23, 4, 0, tzinfo=TzInfo(UTC)),
    'trade_count': 359923.0,
    'volume': 29150091.0,
    'vwap': 177.089781}, {   'close': 175.5,
    'high': 177.4682,
    'low': 174.3,
    'open': 176.7,
    'symbol': 'AMZN',
    'timestamp': datetime.datetime(2024, 8, 26, 4, 0, tzinfo=TzInfo(UTC)),
    'trade_count': 337930.0,
    'volume': 22366236.0,
    'vwap': 175.698201}, {   'close': 173.12,
    'high': 174.89,
    'low': 172.25,
    'open': 174.15,
    'symbol': 'AMZN',
    'timestamp': datetime.datetime(2024, 8, 27, 4, 0, tzinfo=TzInfo(UTC)),
    'trade_count': 377150.0,
    'volume': 29841979.0,
    'vwap': 173.226075}, {   'close': 170.8,
    'high': 173.69,
    'low': 168.92,
    'open': 173.69,
    'symbol': 'AMZN',
    'timestamp': datetime.datetime(2024, 8, 28, 4, 0, tzinfo=TzInfo(UTC)),
    'trade_count': 382296.0,
    'vol

In [2]:
import os
import pandas as pd
from psycopg_pool import ConnectionPool
from datetime import datetime
import turtle
from turtle.service.data_update import DataUpdate


In [3]:
print(turtle)

<module 'turtle' from '/home/jack/github/turtle-backtest/turtle/__init__.py'>


In [ ]:
from datetime import datetime
from turtle.service.data_update import DataUpdate
end_date: datetime = datetime(year=2024, month=9, day=13)
data_updater = DataUpdate()
data_updater.darvas_box_strategy.collect_historical_data('ANNX', end_date, end_date)
# print(data_updater.darvas_box_strategy.df.info())
# data_updater.darvas_box_strategy.df = data_updater.darvas_box_strategy.df.reset_index()
data_updater.darvas_box_strategy.darvas_box_breakout()
print(data_updater.darvas_box_strategy.df.info())
print(data_updater.darvas_box_strategy.df)

In [1]:
!pip3 list

Package                   Version         Editable project location
------------------------- --------------- ---------------------------------
alpaca-py                 0.28.3
altair                    5.4.1
annotated-types           0.7.0
anyio                     4.4.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     24.2.0
babel                     2.16.0
beautifulsoup4            4.12.3
bleach                    6.1.0
blinker                   1.8.2
cachetools                5.5.0
certifi                   2024.8.30
cffi                      1.17.1
charset-normalizer        3.3.2
click                     8.1.7
comm                      0.2.2
debugpy                   1.8.5
decorator                 5.1.1
defusedxml                0.7.1
executing                 2.1.0
fastjsonschema            2.20.0
fqdn                      1.5.1
frozendict  